In [1]:
%matplotlib inline

In [3]:
# Author: Yulou Zhou

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv
import glob
import os
import pandas as pd
import numpy as np
import csv
from random import shuffle
torch.manual_seed(1)

In [4]:
path = ''
note = {}
files = glob.glob(os.path.join(path, '*.csv'))
newvar = []

windows = []
for file in files:
    with open(file, 'r') as f:
        #print(file)
        reader = csv.reader(f)
        all_list = list(reader)
        wsize = 64
        halfsize = wsize / 2
        for i in range(len(all_list) - wsize):
            windpairs = all_list[i: i+wsize]
            x = [element[0] for element in windpairs]
            y = [element[1] for element in windpairs]
            windows.append((x, y))

In [121]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

word_to_ix={}
shuffle(windows)
training_data = windows[:65536]
for sent, tags in windows:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = int(word)
print(word_to_ix)
tag_to_ix = word_to_ix


# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 30

{'10': 10, '8': 8, '7': 7, '5': 5, '3': 3, '12': 12, '2': 2, '0': 0, '9': 9, '1': 1, '4': 4, '11': 11, '6': 6}


Create the model:



In [122]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        #print(sentence)
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [123]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    #print(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    #_, point = tag_scores.max(1)
    #print(point)


for epoch in range(100):  # again, normally you would NOT do 300 epochs, it is toy data
    shuffle(training_data)
    for sentence, tags in training_data[:256]:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
    if(epoch%10==0): print(loss)


# See what the scores are after training
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(inputs)

#     # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#     # for word i. The predicted tag is the maximum scoring tag.
#     # Here, we can see the predicted sequence below is 0 1 2 0 1
#     # since 0 is index of the maximum value of row 1,
#     # 1 is the index of maximum value of row 2, etc.
#     # Which is DET NOUN VERB DET NOUN, the correct sequence!
#     _, point2 = tag_scores.max(1)
#     print(point2)

tensor(2.0329)
tensor(1.6194)
tensor(1.7807)
tensor(1.9433)
tensor(1.3054)
tensor(1.0676)
tensor(3.4759)
tensor(2.2162)
tensor(0.8934)
tensor(1.5421)


In [125]:
accuracy = 0.0
c = 0
for i in range(1000):
    shuffle(training_data)
    with torch.no_grad():
        inputs = prepare_sequence(training_data[0][0][10:20], word_to_ix)
        tag_scores = model(inputs)

        # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
        # for word i. The predicted tag is the maximum scoring tag.
        # Here, we can see the predicted sequence below is 0 1 2 0 1
        # since 0 is index of the maximum value of row 1,
        # 1 is the index of maximum value of row 2, etc.
        # Which is DET NOUN VERB DET NOUN, the correct sequence!
        _, point2 = tag_scores.max(1)
        #print(point2.tolist())

    values, indices = tag_scores.max(1)
    #print(training_data[0][1][10:20])
    #print(training_data[0][0][10:20])
    truth = np.array([int(a) for a in training_data[0][1][10:20]])
    predicted = point2.numpy()
    accuracy += np.average((truth == predicted))
    c += 1
print(accuracy/c)

0.28459999999999985
